In [1]:
!poetry add pandas
!poetry add openmeteo_requests
!poetry add retry_requests

The following packages are already present in the pyproject.toml and will be skipped:

  - pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
Using version ^1.3.0 for openmeteo-requests

Updating dependencies
Resolving dependencies... (1.9s)Resolving dependencies... (1.5s)

Package operations: 1 install, 0 updates, 0 removals

  - Installing sklearn (0.0.post12): Pending...
  - Installing sklearn (0.0.post12): Preparing...
  - Installing sklearn (0.0.post12): Failed

  ChefBuildError

  Backend subprocess exited when trying to invoke get_requires_for_build_wheel
  
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands. 
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' b

In [20]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
import requests

In [9]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [45]:
api_url ="https://archive-api.open-meteo.com/v1/archive"

In [56]:
COORDINATES = {
    "Madrid": {"latitude": 40.416775, "longitude": -3.703790},
    "London": {"latitude": 51.507351, "longitude": -0.127758},
    "Rio": {"latitude": -22.906847, "longitude": -43.172896},
}

VARIABLES = ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max"]

# Define the date range
params = {
    "latitude": COORDINATES["Madrid"]["latitude"],
    "longitude": COORDINATES["Madrid"]["longitude"],
    "start_date": "2024-01-17",
    "end_date": "2024-10-30",
    "hourly": "temperature_2m"  # Single variable for testing
}

# Dictionary to store the combined dataframes for each location
dataframes = {}


In [ ]:
responses = openmeteo.weather_api(api_url,params)


In [59]:
if responses:
    response = responses[0]
    print("Data for Madrid:")
    print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation: {response.Elevation()} m asl")
    print(f"Timezone: {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()} s")

    # Process hourly data for temperature_2m_mean
    hourly = response.Hourly()
    dates = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )
    
    # Create a DataFrame with dates and temperature values
    madrid_data = pd.DataFrame({
        "date": dates,
        "temperature_2m_mean": hourly.Variables(0).ValuesAsNumpy()
    })
    
    print(madrid_data)
else:
    print("Error fetching data for Madrid")

Data for Madrid:
Coordinates: 40.38664245605469°N -3.67608642578125°E
Elevation: 666.0 m asl
Timezone: None None
Timezone difference to GMT+0: 0 s
                          date  temperature_2m_mean
0    2024-01-17 00:00:00+00:00            11.005000
1    2024-01-17 01:00:00+00:00            11.105000
2    2024-01-17 02:00:00+00:00            11.155001
3    2024-01-17 03:00:00+00:00            11.255000
4    2024-01-17 04:00:00+00:00            11.755000
...                        ...                  ...
6907 2024-10-30 19:00:00+00:00            14.655001
6908 2024-10-30 20:00:00+00:00            13.105000
6909 2024-10-30 21:00:00+00:00            12.055000
6910 2024-10-30 22:00:00+00:00            12.305000
6911 2024-10-30 23:00:00+00:00            12.255000

[6912 rows x 2 columns]
